<h1> 실제 작동하는 부분은 10번셀부터이다 </h1>

In [2]:
import pandas as pd

df_satin = pd.read_excel("./data/2022년_공단직영_상위_충전량_10위_요약.xlsx")
df_satin

,주소,addr,addr(로드뷰로_확인전),위도(Latitude),경도(Longitude),충전량(kWh),특이사항
0,시청별관,정확한 주소 없음,연암로 60 (입구50m 우측),35.890672,128.600535,97547.18,NaN
1,농수산물도매시장,대구광역시 북구 매천동 526,매천로18길 34 (4문 좌측),35.901915,128.541006,82014.83,NaN
2,농수산물도매시장2,대구 북구 매천로18길 34,매천로18길 34 (4문 좌측),35.901794,128.541782,82014.83,NaN
3,아양아트센터,대구 동구 효동로2길 24,효동로 2길 24,35.879509,128.652736,77083.54,NaN
4,동구청,대구 동구 아양로 207 동구청 (신암동 36-1),아양로 207 (입구 우측),35.886491,128.635772,76874.81,주차장에 충전소가 있는데 나무때문에 그늘짐.
5,북구청,대구 북구 옥산로 65,옥산로 65 (별관 입구 좌측),35.884923,128.583276,68311.98,NaN
6,충혼탑주차장,대구 남구 대명동 417-52,앞산순환로 540 (입구 좌측),35.832754,128.585479,64537.74,NaN
7,달서구청,대구 달서구 학산로 45 달서구청 (월성동 281),학산로 45 (입구 좌측),35.829420,128.533027,62135.23,주차장에 충전소가 있는데 나무때문에 그늘짐.
8,대구공공시설관리공단(지산),무학로 112 (입구 좌측),무학로 112 (입구 좌측),35.829637,128.621347,60740.71,NaN
9,파동교네거리주차장,정확한 주소 없음,파동 274 (입구 우측),35.810031,128.618947,60564.27,NaN


In [3]:
import requests
import ssl
from requests.adapters import HTTPAdapter


# SSL/TLS 컨텍스트 설정 변경
class SSLAdapter(HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_context'] = self.ssl_context
        return super().init_poolmanager(*args, **kwargs)

ssl_context = ssl.create_default_context()
ssl_context.set_ciphers('DEFAULT:@SECLEVEL=1')

adapter = SSLAdapter(ssl_context=ssl_context)
session = requests.Session()
session.mount('https://', adapter)


In [4]:
import pandas as pd

# 시계열 일사량 api로 데이터를 가져오는 함수
def get_solar_api_data(latitude, longitude, date):

    # URL에 변수 삽입
    url = f"https://apis.data.go.kr/B551184/openapi/service/SolarGhiService/getSolarGhiHrInfo?serviceKey=KmJehGYlsJSeUOwlr80UIbVByWokvs7BEY0bv5sRkdAR6mrcK2TDWWf6UAbukCfhxUuCG4ktHBpriZPXYwbS0g%3D%3D&pageNo=1&numOfRows=50&date={date}&lat={latitude}&lon={longitude}&type=json"

    
    response = session.get(url)
    # print(response.text)  # 디버깅 코드


    # JSON 응답을 파싱
    data = response.json()
    return data

# 시계열 일사량를 데이터프레임으로 만드는 함수
def solar_data_to_df(data, name):
    # 필요한 데이터 추출
    items = data['response']['body']['items']['item']

    # 데이터프레임으로 변환
    df = pd.DataFrame(items)

    # name 열 추가 및 name 변수 값 삽입
    df.insert(0, 'name', name)  # 첫 번째 열로 name 열을 삽입

    # 데이터프레임 리턴
    return df

In [5]:
from datetime import datetime, timedelta

# 특정 월의 모든 날짜를 YYYYMMDD 형식의 문자 리스트로 반환하는 함수
def date_month_generation(year, month):
    start_date = datetime(year, month, 1)
    if month == 12:
        end_date = datetime(year + 1, 1, 1)
    else:
        end_date = datetime(year, month + 1, 1)
    date_generated = [(start_date + timedelta(days=x)).strftime('%Y%m%d') for x in range(0, (end_date - start_date).days)]
    return date_generated

In [28]:
# 특정 월의 데이터를 수집하는 함수
def gather_2022_solar_data_for_a_month(df_satin, real_lat, real_lon, search_month):
    year = 2022
    date_month = date_month_generation(year, search_month)
    df_solor = pd.DataFrame()   # df_solar 초기화
    name = df_satin['주소'][0]

    for date in date_month:
        data_solar = get_solar_api_data(real_lat, real_lon, date)
        
        if data_solar and data_solar['response']['body']['items']:
            df_daily = solar_data_to_df(data_solar, name)   # 하루 df 만듦
            df_solor = pd.concat([df_solor, df_daily], ignore_index=True)  # df_solar에 하루 df 추가
            print(f"{date} 데이터 처리 완료")
        else:
            print(f"{name} : {date}의 데이터는 비어있음")
    
    return df_solor

In [22]:
import math


# api에 좌표를 입력하면 입력좌표와 가까운 여러 좌표가나오므로 가까운 좌표를 알아내야함
# 입력좌표와 실제좌표가 가장 가까운 실제좌표를 리턴하는 함수
def find_nearest_data(user_lat, user_lon, data_items):
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0  # 지구 반지름 (킬로미터)
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)
        a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
        return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    nearest_item = min(data_items, key=lambda item: haversine(user_lat, user_lon, float(item['lat']), float(item['lon'])))
    nearest_distance = haversine(user_lat, user_lon, float(nearest_item['lat']), float(nearest_item['lon']))
    
    print(f"입력 좌표: ({user_lat}, {user_lon})")
    print(f"실제 좌표: ({nearest_item['lat']}, {nearest_item['lon']})")
    print(f"거리: {nearest_distance:.2f} km")
    
    return nearest_item['lat'], nearest_item['lon']

<h2> 사용방법 : </h2>
<h2> 1) find_nearest_data함수로 입력 좌표와 가장 가까운 좌표를 찾는다. </h2>
<h2> 2) 좀전에 찾은 값을 gather_2022_solar_data_for_a_month함수에 넣고 한달씩 데이터를 가져온다. </h2>

In [23]:
# 입력좌표와 가장 가까운 실제좌표를 찾는 코드!

user_lat, user_lon = df_satin['위도(Latitude)'][0], df_satin['경도(Longitude)'][0]
data_solar = get_solar_api_data(user_lat, user_lon, 20220102)   # 20220102의 데이터가 없으면 다른 값 넣기!

data_items = data_solar['response']['body']['items']['item']

real_lat, real_lon = find_nearest_data(user_lat, user_lon, data_items)


입력 좌표: (35.8906723, 128.6005347)
실제 좌표: (35.8917331287, 128.6046379933)
거리: 0.39 km


In [26]:
df_satin_solor = pd.DataFrame()  # 공단 일사량 데이터프레임 만듦. 초기화되는 코드이니 조심

In [27]:
df_satin_solor_backup = df_satin_solor   # 함수 실행전 백업은 필수!
df_satin_solor_backup

""


In [29]:
search_month = 1
df_solor = gather_2022_solar_data_for_a_month(df_satin, real_lat, real_lon, search_month)

df_satin_solor = pd.concat([df_satin_solor, df_solor], ignore_index=True)


시청별관 : 20220101의 데이터는 비어있음
20220102 데이터 처리 완료
20220103 데이터 처리 완료
20220104 데이터 처리 완료
20220105 데이터 처리 완료
20220106 데이터 처리 완료
20220107 데이터 처리 완료
20220108 데이터 처리 완료
20220109 데이터 처리 완료
20220110 데이터 처리 완료
20220111 데이터 처리 완료
20220112 데이터 처리 완료
20220113 데이터 처리 완료
20220114 데이터 처리 완료
20220115 데이터 처리 완료
20220116 데이터 처리 완료
20220117 데이터 처리 완료
20220118 데이터 처리 완료
20220119 데이터 처리 완료
20220120 데이터 처리 완료
20220121 데이터 처리 완료
20220122 데이터 처리 완료
20220123 데이터 처리 완료
20220124 데이터 처리 완료
20220125 데이터 처리 완료
20220126 데이터 처리 완료
20220127 데이터 처리 완료
20220128 데이터 처리 완료
20220129 데이터 처리 완료
20220130 데이터 처리 완료
20220131 데이터 처리 완료


In [31]:
df_satin_solor.to_excel("test.xlsx")